In [ ]:
import numpy as np
import os
import h5py

import mspt.image_processing as img
import mspt.particle_detection as detect
import mspt.particle_fitting as fit
os.environ['NUMEXPR_MAX_THREADS'] = '32'
%matplotlib notebook

# Process movie for display

## Select movie to process

In [ ]:
#file = os.path.normpath(r'C:\Users\admin\Desktop\MSPT test\0001_sample.mp') # set manually
file = img.fileDialog(os.getcwd()) # or via dialog
assert os.path.isfile(file), 'File "{}" does not exist'.format(file)

## Parameter settings

In [ ]:
# General parameters
frame_range = [0,10000] # Restrict analysis to certain frames, e.g. [0, 2000], only applied if all_frames = False
navg = 1 # Frame averaging before background removal
assert len(frame_range)==2 or len(frame_range)==0, 'frame_range is expected to be either of type [] or [int, int]'

# Background removal
mode = 'continuous_median' # Choose background removal strategy
window_length = 1001 # Set median window length

# Choose between CPU and GPU processing. Set one of the two options True and the other one False.
parallel = True # Use multiple CPU cores to perform background substraction
GPU = False # Use GPU to perform background substraction, fastest option (requires CUDA and pytorch)

## Background removal

In [ ]:
# Apply continuous median background removal
frames, file = img.mp_reader(batch_mode=True,
                             file_to_load=file,
                             frame_range=frame_range,
                             mode=mode,
                             navg=navg,
                             window_length=window_length,
                             parallel=parallel, 
                             GPU=GPU)

# Alternatively, load processed movie for display

## Select processed movie to load

In [ ]:
#file = os.path.normpath(r'C:\Users\admin\Desktop\MSPT test\001_sample_median1001.h5') # set manually
file = img.fileDialog(os.getcwd()) # or via dialog
assert os.path.isfile(file), 'File "{}" does not exist'.format(file)

# Load processed movies from HDF5 file
with h5py.File(file, 'r') as h5_file:
    frames = np.asarray(h5_file['frames']).copy()
    window_length = np.asarray(h5_file['window_length']).copy()
    frame_range = np.asarray(h5_file['frame_range']).copy()
    
print('Loaded processed movie {}'.format(file))

# Movie viewer

## Basic viewer with frame slider

In [ ]:
# Use this viewer to slide through frames
contrast_range = [-0.0075, 0.0075] # For display
img.frame_slider(frames, vmin=contrast_range[0], vmax=contrast_range[1], figsize=(9.5, 9.5*frames.shape[1]/frames.shape[2]))

## Viewer with frame slider and detection threshold selection

In [ ]:
# Use this viewer to select a suitable particle detection threshold thresh
contrast_range = [-0.0075, 0.0075] # For display
detect.frame_slider_view_cands(frames,
                               vmin=contrast_range[0],
                               vmax=contrast_range[1],
                               figsize=(9.5, 9.5*frames.shape[1]/frames.shape[2]) )

## Viewer with frame slider, detection threshold selection and successful particle fits

In [ ]:
# Use this viewer to see successful particle fits
# NOTE: takes some time after first execution of cell and first slider update, but then it is smooth
contrast_range = [-0.0075, 0.0075] # For display
fit.frame_slider_view_cands_dets(frames,
                                 vmin=contrast_range[0],
                                 vmax=contrast_range[1],
                                 method='trust-ncg',
                                 DoG_estimates={'T' : 0.1423, 's' : 2.1436, 'sigma' : 1.2921},
                                 figsize=(9.5, 9.5*frames.shape[1]/frames.shape[2]) )